In [1]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

from sklearn.manifold import TSNE
import plotly.express as px


/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [3]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [4]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [5]:
with open('data/ram_tag_list.txt', 'r') as f:
    check=f.readlines()

In [6]:
check=[x.split('\n')[0] for x in check]

In [7]:
maf=check

In [8]:
call=iab_labels+maf

In [9]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [10]:
total_embeddings=[]

In [11]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [16]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

100%|██████████| 4745/4745 [10:53<00:00,  7.26it/s]  


In [17]:
len(total_embeddings)

9144

In [18]:
iab_embeddings=total_embeddings[0:160]   #Change this if you want to test a new external keyword set

In [19]:
maf_embeddings=total_embeddings[160:]

In [20]:
nodes=maf

In [21]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.76887481]]


In [22]:
with open("data/problematic.pkl", 'rb') as f:
    problematic=pkl.load(f)

In [23]:
with open("data/child_parent_dic.pkl", 'rb') as f:
    child_parent_dic=pkl.load(f)

In [24]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)

    #low initial threshold
    similarities=[x for x in similarities if x[2]>=0.70]
    new_similarities=[]


        
    

    # #higher threshold for problematic keywords, high threshold for general keywords for accuracy
    for x in similarities:
        if len(new_similarities)>=15 and x[2]<0.82:  #TOP N Filtering
            break
        if x[2]>=0.82 and x[1] in problematic:   # Problematic Keyword Threshold
            new_similarities.append(x)
        elif x[2]>=0.80 and x[1] not in problematic:  # Non Problematic Threshold
            new_similarities.append(x)
        else:
            continue
    
    #adding atleast 1 adword
    if len(new_similarities)==0:
        while len(new_similarities)<1:
            new_similarities.append(similarities[len(new_similarities)])
        



    #force adding all parent adwords if child is predicted
    new_sim_adwords=[]
    for items in new_similarities:
        new_sim_adwords.append(items[1])

    count=0
    for items in new_similarities:
        if count==5:
            break
        try:
            parent_list=child_parent_dic[items[1]]
            for parent in parent_list:
                if parent not in new_sim_adwords:
                    new_sim_adwords.append(parent)
                    j=call.index(parent)
                    cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
                    if cos_sim>0.765: #Only adding parent if cosine similarity is above 0.765
                        new_similarities.append([call[160+i],call[j],cos_sim])
        except Exception as e:
            #print(e)
            pass

        count+=1
    


        
    temp_iab_labels=[]
    temp_cosines=[]

    count=0
    for items in new_similarities:
        if count>=15 and items[2][0][0]<0.82:
            break
        tsne_embeddings.append(maf_embeddings[i])

        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0][0])
        top_n_nodes.append(nodes[i])

        count+=1

    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)

 51%|█████     | 4585/8984 [09:19<08:57,  8.19it/s]


IndexError: list index out of range

In [25]:
len(top_n_labels_new) #Number of mappings

33257

In [26]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [27]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
#df_MAF_new.to_csv('results/RAM_results_0.80_TSNE_Mapping.csv')

In [ ]:
#CALCULATING STATISTICS BELOW

In [29]:
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print("Number of adwords predicted per keyword")
print(count/len(maf))

100%|██████████| 4585/4585 [00:21<00:00, 213.78it/s]

Number of adwords predicted per keyword
7.253435114503817


In [30]:
#Creating Adword Statistics File
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

100%|██████████| 160/160 [00:00<00:00, 241.42it/s]


In [31]:
iab_labels[counts.index(max(counts))] #Most Used

'Style & Fashion'

In [32]:
iab_labels[counts.index(min(counts))] #Least Used

'Station Wagon'

In [33]:
sum(counts)/160

207.85625

In [34]:
print(count/len(maf))

7.253435114503817
